# Dataset devision

In [51]:
import torch
from torch import optim as optim

In [52]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])

In [53]:
t_u.shape

torch.Size([11])

In [54]:
n_samples = t_u.shape[0]
n_samples

11

In [55]:
# Размер валидационной выборки
n_val = int(0.2 * n_samples)
n_val

2

In [56]:
# Перемешиваем индексы
shuffled_indices = torch.randperm(n_samples)
shuffled_indices

tensor([ 8,  0,  5,  7,  2, 10,  4,  1,  3,  9,  6])

In [57]:
# Получаем индексы для обучающей и валидационной выборок
train_indices = shuffled_indices[: -n_val]
val_indices = shuffled_indices[-n_val :]

print(f"train_indices: {train_indices}")
print(f"val_indices: {val_indices}")

train_indices: tensor([ 8,  0,  5,  7,  2, 10,  4,  1,  3])
val_indices: tensor([9, 6])


In [58]:
# Получаем входные данные и метки для обучащей выборки:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

print(f"train_t_u: {train_t_u}")
print(f"train_t_c: {train_t_c}")

train_t_u: tensor([48.4000, 35.7000, 48.9000, 21.8000, 58.2000, 68.4000, 56.3000, 55.9000,
        81.9000])
train_t_c: tensor([ 6.0000,  0.5000,  8.0000, -4.0000, 15.0000, 21.0000, 11.0000, 14.0000,
        28.0000])


In [59]:
# Получаем входные данные и метки для валидационной выборки:
val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

print(f"val_t_u: {val_t_u}")
print(f"val_t_c: {val_t_c}")

val_t_u: tensor([60.4000, 33.9000])
val_t_c: tensor([13.,  3.])


In [60]:
# Нормализация
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

## Цикл обучения

In [61]:
def model(t_u, w, b):
    return w * t_u + b

In [62]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [63]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <=3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
    return params        


In [64]:
params = torch.tensor([0.1, 0.0], requires_grad=True)
learning_rate = 1e-2
# Optimizer creating:
optimizer = optim.SGD([params], lr=learning_rate)

In [65]:
training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    params=params,
    train_t_u=train_t_un,
    val_t_u=val_t_un,
    train_t_c=train_t_c,
    val_t_c=val_t_c
)

Epoch 1, Training loss 194.8522, Validation loss 80.3709
Epoch 2, Training loss 55.8134, Validation loss 9.4561
Epoch 3, Training loss 36.7419, Validation loss 8.7582


Epoch 500, Training loss 8.1423, Validation loss 2.1599
Epoch 1000, Training loss 3.4706, Validation loss 2.9242
Epoch 1500, Training loss 2.6288, Validation loss 4.3674
Epoch 2000, Training loss 2.4771, Validation loss 5.1816
Epoch 2500, Training loss 2.4497, Validation loss 5.5636
Epoch 3000, Training loss 2.4448, Validation loss 5.7322


tensor([  5.5788, -18.4097], requires_grad=True)